The purpose of this notebook is to load all the loose txt files into a single dataframe saved in the "data.csv" file. 

This notebook:
1. Loads all the texts (.txt) into a pandas DataFrame.
2. Loads all the corresponding gold labels into the DataFrame.
3. Encodes those labels and saves the encodings.
4. Translates all non-english texts into english.

Now that's all clear, let's import what we will need in the notebook.

In [1]:
import os
import pandas as pd
import numpy as np
import nltk
import pickle

from deep_translator import GoogleTranslator
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.tokenize import sent_tokenize

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/jochem/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Now that we have all the relevant packages, lets start of with point 1: loading all .txt files into a data frame. 

So, let's describe a function that loops through a directory, reads the text files and puts those text files into a DataFrame.

Next, we load the data from all the directories.

In [2]:
def load_data(directory:str):
    data = []

    for file in os.listdir(directory):
        file_path = os.path.join(directory,file)
        
        english = 0
        if file[:2] == "EN":
            english = 1

        try:
            with open(file_path, 'r') as f:
                text = f.read()
                data.append({"id":file, "text": text, "english": english})
        except Exception as e:
            print(f"Error while reading:{file}")
    
    print("Data length: ", len(data))
    df_text = pd.DataFrame(data)
    return df_text
        


In [3]:
df_text_en = load_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/EN/raw-documents")
df_text_bg = load_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/BG/raw-documents")
#df_text_hi = load_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/HI/raw-documents")
df_text_pt = load_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/PT/raw-documents")
df_text = pd.concat([df_text_en, df_text_bg, df_text_pt]) #df_text_hi,
print(len(df_text))

Data length:  200
Data length:  211
Data length:  200
611


We still need the correct labels for these files though. 

To get these we: 
1. Define a function that goes through the subtask 2 annotation file and loads all the appropriate labels into a DataFrame.
2. Load in all different languages labels.
3. Encode the labels with a MultiLabelBinarizer.
4. Save these in the DF as well. 

In [4]:
def load_labels(file_path:str):
    labels = []
    with open(file_path, "r") as f:
        for line in f:
            tags = line.strip().split("\t")
            
            text_id = tags[0].strip()

            dom_narrs = [narr.strip() for narr in tags[1].split(";")]
            sub_narrs = [narr.strip() for narr in tags[2].split(";")]

            labels.append({"id":text_id,"dom_narr": dom_narrs, "sub_narr": sub_narrs})
    print("Labels length:", len(labels))
    df_labels = pd.DataFrame(labels)
    return df_labels

In [5]:
df_labels_en = load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/EN/subtask-2-annotations.txt")
df_labels_bg = load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/BG/subtask-2-annotations.txt")
#df_labels_hi = load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/HI/subtask-2-annotations.txt")
df_labels_pt = load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/PT/subtask-2-annotations.txt")
df_labels = pd.concat([df_labels_en, df_labels_bg, df_labels_pt]) #df_labels_hi,
print(len(df_labels))

Labels length: 200
Labels length: 211
Labels length: 200
611


Now for the encoding:
1. Again, we define a function to do this for us.
2. Encode the labels and save them in the DF.
3. Save the MultiLabelBinarizer objects in a pickle (.pkl) file, so we can access them later, to revert predictions back into full classes.

In [6]:
# label preprocessing steps
# one-hot encodes the dominant- and sub-narrative labels       
# TODO: also make this function save the dom and sub mlbs       
def encode_labels(df, save:bool=True):
    dom_mlb = MultiLabelBinarizer()
    sub_mlb = MultiLabelBinarizer()

    dom_narr_enc = dom_mlb.fit_transform(df["dom_narr"])
    df = pd.concat([df, pd.DataFrame(dom_narr_enc, columns=dom_mlb.classes_)], axis=1)

    sub_narr_enc = sub_mlb.fit_transform(df["sub_narr"])
    df = pd.concat([df, pd.DataFrame(sub_narr_enc, columns=sub_mlb.classes_)], axis=1)
    
    if(save):
        with open("../pkl_files/dom_mlb.pkl", "wb") as f:
            pickle.dump(dom_mlb,f)
        
        with open("../pkl_files/sub_mlb.pkl", "wb") as f:
            pickle.dump(sub_mlb,f)

    return df

In [7]:
df = encode_labels(pd.merge(df_text, df_labels, on="id"))
df.to_clipboard()
print(len(df))

611


Next, we would like to translate the non-english texts into english. So,

1. Define a function to translate a list of strings into english.
2. Apply this function to all non english texts.
3. While applying the translation function, we call nltk's sent_tokenizer, to split the texts up into an array of sentence strings.

In [8]:
# source lang = "auto", target lang = "en"
translator = GoogleTranslator()
def translate_text(text:list[str]):
    translation = []
    
    try:
        translation = [translator.translate(t) for t in text]
        translation = [t for t in translation if t is not None]
        translation = " ".join(translation)

    except Exception as error:
        print("translation failed")
        print(error)
        
    return translation

In [9]:
# only using sent tokenize and translating each sentence individually had some serious performance issues,
# so now lets chunk the text to a length thats doable.
def chunk_text(text:str, chunk_size=3500) -> list[str]:
    if(len(text) < chunk_size):
        return [text]
    
    chunks = []
    current_chunk = []
    current_chunk_length = 0

    sentence_tokens = sent_tokenize(text)
    for sent in sentence_tokens:
        sent_length = len(sent.encode('utf-8'))

        if (sent_length + current_chunk_length) >= chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sent]
            current_chunk_length = sent_length + 1
        
        else:
            current_chunk.append(sent)
            current_chunk_length += sent_length + 1
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks



In [10]:
chunked_text = df.loc[(df["english"] == 0), "text"].apply(lambda x: chunk_text(x))
chunked_text.to_clipboard()

BE AWARE THE FOLLOWING CELL TAKES A LONG TIME TO COMPLETE!

It applies the translation to all the non english texts.
Expect to wait at least 10 minutes for the result.

In [11]:
df.loc[(df["english"] == 0), "text"] = df.loc[(df["english"] == 0), "text"].apply(lambda x: translate_text(chunk_text(x)))

translation failed
Request exception can happen due to an api connection error. Please check your connection and try again
translation failed
Request exception can happen due to an api connection error. Please check your connection and try again
translation failed
Request exception can happen due to an api connection error. Please check your connection and try again
translation failed
Request exception can happen due to an api connection error. Please check your connection and try again
translation failed
Request exception can happen due to an api connection error. Please check your connection and try again


In [17]:
print(len(df))
df.to_clipboard()

611


The last thing we want to do is save our full data set to a csv file. 

In [13]:
df.to_csv("../data/newdata.csv")

In [18]:
amount =0 
for i in range(len(df)):
    if(df["text"][i] == []):
        print(df["id"][i])
        amount += 1

print(amount)

BG_971.txt
BG_1249.txt
BG_613.txt
BG_561.txt
BG_1637.txt
5


In [15]:
problematic_text = """'पापा ने वॉर रुकवा दी' से लेकर ग्लोबल पीसमेकर तक, जानिए कैसे रूस और ...

'पापा ने वॉर रुकवा दी' से लेकर ग्लोबल पीसमेकर तक, जानिए कैसे रूस और यूक्रेन के बीच संतुलन बना रहे पीएम मोदी?
By मनाली रस्तोगी | Published: August 30, 2024 07:25 AM2024-08-30T07:25:07+5:302024-08-30T07:41:48+5:30
रूस और यूक्रेन दोनों के साथ भारत के ऐतिहासिक संबंधों के साथ मिलकर इस संतुलन अधिनियम ने इन अशांत समय में शांतिदूत के रूप में पीएम मोदी की क्षमता के बारे में उम्मीदें जगाई हैं। वह सही तरीके से कार्ड खेल रहे हैं।
रूस और यूक्रेन के बीच चल रहे संघर्ष के बीच प्रधानमंत्री नरेंद्र मोदी की कूटनीतिक चाल ने भारत को वैश्विक मंच पर एक महत्वपूर्ण खिलाड़ी के रूप में स्थापित किया है। मोदी की हालिया यूक्रेन यात्रा, जो तीन दशकों में किसी भारतीय प्रधान मंत्री की पहली यात्रा है, ने न केवल भारत में चर्चा को फिर से शुरू कर दिया है, बल्कि परस्पर विरोधी वैश्विक शक्तियों के बीच एक नाजुक संतुलन बनाए रखने की उनकी क्षमता का भी प्रदर्शन किया है।
रूस और यूक्रेन दोनों के साथ भारत के ऐतिहासिक संबंधों के साथ मिलकर इस संतुलन अधिनियम ने इन अशांत समय में शांतिदूत के रूप में पीएम मोदी की क्षमता के बारे में उम्मीदें जगाई हैं। वह सही तरीके से कार्ड खेल रहे हैं। चाहे वह भारत द्वारा आयोजित जी20 शिखर सम्मेलन हो, या छह सप्ताह के अंतराल में रूस और यूक्रेन दोनों की लगभग बैक-टू-बैक यात्राएं हों।
कूटनीतिक रणनीति
रूस के साथ भारत के संबंध शीत युद्ध के युग से हैं जब सोवियत संघ एक दृढ़ सहयोगी था। 1971 में हस्ताक्षरित शांति, मित्रता और सहयोग की भारत-सोवियत संधि दोनों देशों के बीच गहरे संबंधों का प्रमाण है।
रूस एक महत्वपूर्ण रक्षा भागीदार रहा है, जो दशकों से भारत को सैन्य उपकरण और प्रौद्योगिकी की आपूर्ति कर रहा है। दूसरी ओर, सोवियत संघ से स्वतंत्रता प्राप्त करने के बाद यूक्रेन भी भारत के लिए एक महत्वपूर्ण भागीदार रहा है, खासकर रक्षा, शिक्षा और कृषि जैसे क्षेत्रों में।
रूस-यूक्रेन संघर्ष के दौरान इन रिश्तों को निभाना एक कठिन चुनौती रही है। जबकि अमेरिका के नेतृत्व में पश्चिम ने देशों पर रूस के खिलाफ रुख अपनाने के लिए दबाव डाला है, वहीं भारत ने बातचीत और शांति की वकालत करते हुए तटस्थ रुख बनाए रखा है।
मोदी सरकार ने रूस और यूक्रेन दोनों के साथ बातचीत जारी रखी है और रूस को अलग-थलग करने के अंतरराष्ट्रीय दबाव के आगे झुकने से इनकार कर दिया है। इसका उदाहरण पश्चिमी प्रतिबंधों के बावजूद कच्चे तेल की खरीद सहित रूस के साथ भारत के चल रहे व्यापारिक सौदे हैं।
मोदी की यूक्रेन यात्रा प्रतीकात्मक थी, जो शांति के प्रति भारत की प्रतिबद्धता और इसमें शामिल सभी पक्षों के साथ जुड़ने की इच्छा का संकेत देती है। रूस और यूक्रेन दोनों में उन्हें जो स्वागत मिला, वह एक ऐसे राष्ट्र के रूप में भारत की अद्वितीय स्थिति को उजागर करता है जो पूर्व और पश्चिम के बीच की खाई को पाट सकता है। यह तथ्य कि मोदी दोनों देशों में इतना गर्मजोशी से स्वागत करने में कामयाब रहे, यह अंतरराष्ट्रीय मंच पर उनके सम्मान को उजागर करता है।
सोशल मीडिया पर वायरल हो रहा है 'पापा ने वार रुकवा दी'
भारत में मोदी की यूक्रेन यात्रा से सोशल मीडिया पर टिप्पणियों की लहर दौड़ गई है। 'पापा ने वार रुकवा दी' वाक्यांश, जो लोकसभा चुनाव के दौरान उत्पन्न हुआ था, इस बार एक अलग स्वर के साथ फिर से सामने आया है।
मोदी के समर्थकों ने उनके कूटनीतिक प्रयासों की प्रशंसा करने के लिए एक्स जैसे मंच का सहारा लिया है, ऐसी टिप्पणियों के साथ कि "मोदी जी वास्तव में एक वैश्विक नेता हैं" और "शांतिदूत के रूप में भारत की स्थिति अब मजबूत हो गई है"।
हालांकि, आलोचक संशय में हैं। कुछ लोगों ने सवाल किया है कि क्या मोदी की यात्रा का ज़मीनी स्तर पर कोई ठोस प्रभाव पड़ेगा, "क्या यह यात्रा सिर्फ एक पीआर अभ्यास है?" और "भारत शांति स्थापित करने के लिए क्या ठोस कदम उठा रहा है?"
लेकिन एक बात स्पष्ट है। पीएम मोदी ने जो कर दिखाया, कोई अन्य राष्ट्र प्रमुख नहीं कर पाया। और वो है- महज कुछ हफ्तों के अंतराल में पुतिन और ज़ेलेंस्की दोनों से मुलाकात। यह वैश्विक शांति निर्माता के रूप में भारत की भूमिका और स्थिति को मजबूत करता है।
शांति करने वाला
रूस और यूक्रेन दोनों के साथ जुड़ने के मोदी के प्रयास और बातचीत और शांति पर उनका जोर भारत को संघर्ष में संभावित मध्यस्थ के रूप में स्थापित कर सकता है। भारत की ऐतिहासिक गुटनिरपेक्ष नीति, तटस्थता के वर्तमान रुख के साथ मिलकर, इसे मध्यस्थ के रूप में कार्य करने के लिए आवश्यक विश्वसनीयता प्रदान करती है।
इसके अलावा वैश्विक मंच पर भारत के बढ़ते प्रभाव और रूस और यूक्रेन दोनों के नेताओं के साथ मोदी के व्यक्तिगत तालमेल से भारत दोनों पक्षों को बातचीत की मेज पर लाने में रचनात्मक भूमिका निभा सकता है।
भारत की सामरिक स्वतंत्रता
मोदी के दृष्टिकोण में जो बात विशेष रूप से प्रभावशाली है वह रणनीतिक स्वतंत्रता बनाए रखने की उनकी क्षमता है। रूस के साथ भारत के घनिष्ठ संबंधों और उसकी आर्थिक व्यस्तताओं के बावजूद, मोदी संयुक्त राज्य अमेरिका की अच्छी किताबों में बने रहने में कामयाब रहे हैं।
वर्तमान भू-राजनीतिक माहौल को देखते हुए, यह संतुलन कार्य कोई छोटी उपलब्धि नहीं है, जहां एक शक्ति के साथ बहुत निकटता से जुड़ना अक्सर दूसरों से अलगाव की ओर ले जाता है। मोदी की कूटनीति ने भारत को अपनी संप्रभुता का दावा करने और पूर्व बनाम पश्चिम के द्विआधारी संघर्ष में शामिल हुए बिना अपने राष्ट्रीय हितों को आगे बढ़ाने की अनुमति दी है।
विपक्ष चुप हो गया
दिलचस्प बात यह है कि जहां मोदी की यात्रा ने व्यापक ध्यान आकर्षित किया है, वहीं विपक्ष, विशेषकर कांग्रेस, काफी हद तक चुप रही है। यह चुप्पी उल्लेखनीय है, विशेषकर लोकसभा चुनाव के दौरान विपक्ष द्वारा "पापा ने वार रुकवा दी" तंज के साथ अपनाए गए आक्रामक रुख को देखते हुए।
कांग्रेस की ओर से प्रतिक्रिया की कमी इस बात का संकेत हो सकती है कि मोदी जिस कूटनीतिक रस्सी पर चल रहे हैं, उसकी मान्यता है, जिसकी आलोचना भारत की वैश्विक स्थिति के प्रति गैर-देशभक्त या गैर-समर्थक दिखाई दिए बिना करना मुश्किल है।"""


problematic_chunk = chunk_text(problematic_text)
for chunk in problematic_chunk:
    print (len(chunk))
translated_chunk = translate_text(problematic_chunk[3])

181
3107
60
1874


In [16]:
print(translated_chunk)

L K No  A K  B so  S Fifth note of musical scale Sh T  H  Fifth note of musical scale A I  I The  No  J  K and  The b Y  K E  A No Y  and Sh T and  Fifth note of musical scale and I b  No H  K and  Fifth note of musical scale Y  Oh And  And  H  I H J  K is  H F so  K  A So And L  i  Fifth note of musical scale so No  Oh and  J L S K  The No  S  I L K so  Y H  And Sh And K  Sh so  No And I So  K  and Fifth note of musical scale  i  Bh and So  K  Bh I K  Oh And  S Th so  K  I J B so  K And So  H  Sh So  K And No  And L  And S  Oh And  Y K And No  The No  K  S Th  J D No  K  I The  K  Fifth note of musical scale And Y S  Oh And  B So f So  Oh and  Sh So  Fifth note of musical scale And  U No K  J and  Bh and so  K  S d and Sh  I  S Bh And So  I th Y S Th  K  and Fifth note of musical scale  I  S Th Fifth note of musical scale so  K And  S K So  H  Bh And so  K  Hey so H S K  C T No And Fifth note of musical scale K Sh  No So  so T S Th So  K  And And So I No  and b  K  S Th  i L K And  et